In [ ]:
from src.material.textures.material.checker import CheckerMaterial
from src.material.textures.noise.simplex_noise import SimplexNoise
from src.material.textures.noise.voronoi_noise import VoronoiNoise
from src.render.loops import *
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.material.material.phong_material import PhongMaterial
from src.material.color import Color
from src.geometry.primitives.sphere import Sphere
from src.scene.primitive import Object
from src.scene.scene import Scene
from src.scene.camera import Camera
from src.scene.light import PointLight
from src.math import Vertex, Vector
from src.render.resolution import Resolution

# Simple scene with a sphere and single light source

In [ ]:
from src.scene.light import SpotLight
from src.geometry.primitives.box import Box
from src.material.textures.material.marble import MarbleMaterial
from src.material.textures.noise.ridget_multifractal import RidgeNoise
from src.material.textures.noise.turbulence_noise import TurbulenceNoise
from src import NormalShader, AmbientLight
from src.material.textures.noise.perlin_noise import PerlinNoise
from src.material.textures.noise.FBM import FBMNoise
from src.material.textures.material.rock import RockMaterial
from src.scene import primitive

# mat = PhongMaterial(
#     name="rock",
#     shininess=30,
#     base_color=Color.custom_rgb(255, 200, 200),
#     spec_color=Color.custom_rgb(255, 255, 255),
#     normal_noise=FBMNoise(
#         strength=0.9,
#         scale=3.0,
#         octaves=6,
#         lacunarity=2.1,
#         gain=0.55,
#     )
# )

# white_marble = MarbleMaterial(
#     name="white_marble",
#     base_color=Color.custom_rgb(245, 245, 248),   # warm-ish white
#     spec_color=Color.custom_rgb(220, 220, 220),   # softer highlight (less plastic)
#     shininess=45,                                 # lower than before
# )
#
# white_marble.vein_scale = 7.0        # 5..10
# white_marble.warp_strength = 2.2     # 1.5..3.0
# white_marble.vein_sharpness = 6.0    # thinner veins
#
# white_marble.dark_color_factor = 0.88  # subtle contrast (IMPORTANT)
# white_marble.light_color_factor = 1.0


mat = CheckerMaterial(
    shininess=10,
    reflectivity=0.5,
    base_color=Color.custom_rgb(240,240,240),
    spec_color=Color.custom_rgb(40,40,40),
    scale=4.0
)




# mat = RockMaterial(
#     name="rock",
#     shininess=30,
#     base_color=Color.custom_rgb(255, 200, 200),
#     spec_color=Color.custom_rgb(255, 255, 255),
#     normal_noise=TurbulenceNoise(
#         strength=0.3,
#         scale=1.0,
#         octaves = 5,
#         lacunarity = 2.0,   # frequency multiplier per octave
#         gain = 0.5         # amplitude multiplier per octave
#     )
# )

# mat = PhongMaterial(
#     name="marble_phong",
#     shininess=50,
#     base_color=Color.custom_rgb(220, 220, 220),
#     spec_color=Color.custom_rgb(255, 255, 255),
#     normal_noise=RidgeNoise(
#         strength=0.4,
#         scale=2.0,
#         octaves = 6,
#         lacunarity = 2.0,   # frequency multiplier per octave
#         gain = 0.5         # amplitude multiplier per octave
#     )
# )

#

# mat = PhongMaterial(
#     name="marble_phong",
#     shininess=50,
#     base_color=Color.custom_rgb(220, 220, 220),
#     spec_color=Color.custom_rgb(255, 255, 255),
#     normal_noise=SimplexNoise(
#         scale = 2.0,
#         strength = 0.4,
#         eps = 1e-3
#     )
# )

mat2 = RockMaterial(
    name="rock2",
    shininess=20,
    base_color=Color.custom_rgb(200, 220, 255),
    spec_color=Color.custom_rgb(200, 200, 200),
    color_noise= FBMNoise(
        strength=0.5,
        scale=2.0,
        octaves = 4,
        lacunarity = 2.0,   # frequency multiplier per octave
        gain = 0.5         # amplitude multiplier per octave
    ),
    color_scale=3.0,
    normal_noise=RidgeNoise(
        strength=0.6,
        scale=1.5,
        octaves = 5,
        lacunarity = 2.0,   # frequency multiplier per octave
        gain = 0.5         # amplitude multiplier per octave
    )
)




scene = Scene(
    camera=Camera(
        fov=45,
        aspect_ratio=Resolution.R360p.aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),

    primitives = [
        Object(
            geometry=Box(
                corner1=Vertex(-2.0, -1.0, -8.0),
                corner2=Vertex(2.0, -1.2, -4.0)
            ),
            material=mat,
        ),
        Object(
            geometry=Sphere(
                center=Vertex(0, 0, -6.0),
                radius=1.0
            ),
            material=mat2,
        ),
    ],

    lights=[
        PointLight(
            position=Vertex(5, 5, 0),
            intensity=2000.0,
            falloff=0.01
        ),
        SpotLight(
            position=Vertex(0, 9, -5),
            direction=Vector(0, -1, -0.15),
            angle=0.2,
            intensity=0.9,
            falloff=0.02
        ),
        AmbientLight(
            intensity=0.1
        ),
    ],
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

render_configuration = RenderConfig(
    resolution=Resolution.R480p,
    samples_per_pixel=7,
    max_depth=7,
)


In [ ]:
my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration)
png_path = my_ray_tracer.render("images/rock_full.png")